In [1]:
import $file.common
import spark._
import common._
import org.apache.spark.sql.functions._
import org.apache.spark.rdd._
import org.apache.spark.sql.types.{IntegerType, StringType, StructType}
import org.apache.spark.sql._, types._, functions._
import org.apache.spark.RangePartitioner
import spark.implicits._

Loading spark-stubs
Getting spark JARs


log4j:WARN No appenders could be found for logger (org.eclipse.jetty.util.log).
log4j:WARN Please initialize the log4j system properly.
log4j:WARN See http://logging.apache.org/log4j/1.2/faq.html#noconfig for more info.


Creating SparkSession


Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
21/07/08 10:52:26 INFO SparkContext: Running Spark version 2.4.5
21/07/08 10:52:27 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
21/07/08 10:52:27 INFO SparkContext: Submitted application: Queries Optimization
21/07/08 10:52:27 INFO SecurityManager: Changing view acls to: jovyan
21/07/08 10:52:27 INFO SecurityManager: Changing modify acls to: jovyan
21/07/08 10:52:27 INFO SecurityManager: Changing view acls groups to: 
21/07/08 10:52:27 INFO SecurityManager: Changing modify acls groups to: 
21/07/08 10:52:27 INFO SecurityManager: SecurityManager: authentication disabled; ui acls disabled; users  with view permissions: Set(jovyan); groups with view permissions: Set(); users  with modify permissions: Set(jovyan); groups with modify permissions: Set()
21/07/08 10:52:28 INFO Utils: Successfully started service 'sparkDriver' on port 4

21/07/08 10:52:29 INFO SparkContext: Added JAR file:/home/jovyan/.cache/coursier/v1/https/repo1.maven.org/maven2/org/scala-lang/modules/scala-collection-compat_2.12/2.0.0/scala-collection-compat_2.12-2.0.0-sources.jar at spark://98858bd2f6bb:45503/jars/scala-collection-compat_2.12-2.0.0-sources.jar with timestamp 1625741549337
21/07/08 10:52:29 INFO SparkContext: Added JAR file:/home/jovyan/.cache/coursier/v1/https/repo1.maven.org/maven2/org/scala-lang/modules/scala-collection-compat_2.12/2.0.0/scala-collection-compat_2.12-2.0.0.jar at spark://98858bd2f6bb:45503/jars/scala-collection-compat_2.12-2.0.0.jar with timestamp 1625741549337
21/07/08 10:52:29 INFO SparkContext: Added JAR file:/home/jovyan/.cache/coursier/v1/https/repo1.maven.org/maven2/org/scala-lang/modules/scala-xml_2.12/1.2.0/scala-xml_2.12-1.2.0-sources.jar at spark://98858bd2f6bb:45503/jars/scala-xml_2.12-1.2.0-sources.jar with timestamp 1625741549338
21/07/08 10:52:29 INFO SparkContext: Added JAR file:/home/jovyan/.cache

21/07/08 10:52:29 INFO SparkContext: Added JAR file:/home/jovyan/.cache/coursier/v1/https/repo1.maven.org/maven2/com/fasterxml/jackson/module/jackson-module-scala_2.12/2.9.9/jackson-module-scala_2.12-2.9.9.jar at spark://98858bd2f6bb:45503/jars/jackson-module-scala_2.12-2.9.9.jar with timestamp 1625741549346
21/07/08 10:52:29 INFO SparkContext: Added JAR file:/home/jovyan/.cache/coursier/v1/https/repo1.maven.org/maven2/com/fasterxml/jackson/module/jackson-module-paranamer/2.9.9/jackson-module-paranamer-2.9.9.jar at spark://98858bd2f6bb:45503/jars/jackson-module-paranamer-2.9.9.jar with timestamp 1625741549347
21/07/08 10:52:29 INFO SparkContext: Added JAR file:/home/jovyan/.cache/coursier/v1/https/repo1.maven.org/maven2/com/squareup/okhttp3/logging-interceptor/3.11.0/logging-interceptor-3.11.0.jar at spark://98858bd2f6bb:45503/jars/logging-interceptor-3.11.0.jar with timestamp 1625741549347
21/07/08 10:52:29 INFO SparkContext: Added JAR file:/home/jovyan/.cache/coursier/v1/https/repo1.

Spark UI

import $file.$     

import spark._

import common._

import org.apache.spark.sql.functions._

import org.apache.spark.rdd._

import org.apache.spark.sql.types.{IntegerType, StringType, StructType}

import org.apache.spark.sql._, types._, functions._

import org.apache.spark.RangePartitioner

import spark.implicits._

In [2]:
import plotly._
import plotly.element._
import plotly.layout._
import plotly.Almond._

import plotly._

import plotly.element._

import plotly.layout._

import plotly.Almond._

# Query 1: Media de infecciones diarias por país.

Esta consulta utiliza un único conjunto de datos con las infecciones diarias reportadas por cada país.

El objetivo de la query es:
    - Agrupar el conjunto de datos por paises
    - Sumar todos los datos diarios obteniendo el valor acumulado
    - Dividir el valor acumulado entre el número de días reportados

De esta forma obtendremos la media de infecciones diarias por país.

## 1. RDD para la consulta de infecciones diarias por país

Creo una funcion para trabajar con un RDD de infecciones

In [3]:
def infections(lines : RDD[String]) : RDD[Infection] =
    lines.map(line => {
      val arr = line.split(",")
      Infection(
        day = arr(1).toInt,
        month = arr(2).toInt,
        year = arr(3).toInt,
        nCases = arr(4).toInt,
        nDeaths = arr(5).toInt,
        country = arr(6),
        continent = arr(10)
      )
    })

defined function infections

Calculo la media de infecciones diarias por país trabajando con pair RDD

In [4]:
def infectionGrowthAverage(infections : RDD[Infection]) : RDD[(String, Int)]= {
        infections.map(x => (x.country,x.nCases))
        .groupByKey()
        .mapValues(x => (x.sum / x.size))
        .sortBy(_._2)
  }

defined function infectionGrowthAverage

In [5]:
def infectionGrowthAverageReducedAVG (infections : RDD[Infection]) : RDD[(String, Int)] = {
        infections.map(x => (x.country,(1,x.nCases)))
        .reduceByKey((x,y) => (x._1 + y._1, x._2 + y._2))
        .map{ case(country, (days,cases)) => (country, cases/days) }
        .sortBy(_._2)
}

defined function infectionGrowthAverageReducedAVG

In [6]:
def infectionData = spark.sparkContext.textFile("../datasets/data.csv",8)

defined function infectionData

Cargamos datos y ejecutamos la query.

In [7]:
def infectionRDD = infections(infectionData)

defined function infectionRDD

In [8]:
def query_RDD = infectionGrowthAverage(infectionRDD)

defined function query_RDD

In [9]:
def query_optimized_RDD = infectionGrowthAverageReducedAVG(infectionRDD)

defined function query_optimized_RDD

## 2. Con un DataSet

Definimos el número de particiones en que queremos que nos reparta los datos.

In [10]:
spark.conf.set("spark.sql.shuffle.partitions", 8)

In [11]:
spark.conf.set("spark.sql.files.minPartitionBytes", 8)

In [12]:
def infectionDS = spark.read
    .option("header", "true")
    .csv("../datasets/covidworldwide.csv")
    .as[(String,String,String,String,String,String,String,String,String,String,String,String)]

defined function infectionDS

In [21]:
def query_DS = 
    infectionDS.groupBy($"countriesAndTerritories")
    .agg(avg($"cases"))
    .orderBy("avg(cases)")
    .as[(String,Double)]

defined function query_DS

## 3. Hago los mismos calculos con un DataFrame

#### Creo el DataFrame directamente importando los datos

In [14]:
def dfCovid = spark.read
    .option("header", "true")
    .csv("../datasets/covidworldwide.csv")

defined function dfCovid

Utilizo los métodos de la API DF que incluye uno optimizado para calcular la media.

In [15]:
def query_DF = dfCovid
    .groupBy("countriesAndTerritories")
    .agg(mean("cases"))
    .orderBy("avg(cases)")

defined function query_DF

# 4. Visualización de rendimiento

Gráficamente utilizando plotly

In [23]:
val (x, y) = Seq(
    "RDD" -> runWithOutput(query_RDD.collect),
    "Optimized RDD"-> runWithOutput(query_optimized_RDD.collect),
    "DataSet" -> runWithOutput(query_DS.collect),
    "DataFrame" -> runWithOutput(query_DF.collect)
).unzip

Bar(x, y).plot()

map at cmd3.sc:2

8 / 8

sortBy at cmd3.sc:5

8 / 8

sortBy at cmd3.sc:5

...

8 / 8

collect at cmd22.sc:5

8 / 8

Took 701


x: Seq[String] = List("RDD", "Optimized RDD", "DataSet", "DataFrame")
y: Seq[Int] = List(6968, 497, 671, 701)
res22_1: String = "plot-5264b65f-631f-41e2-ad63-3843ffee1398"

# 5. Comparativas de rendimiento

Usando la API de spark

In [ ]:
val timeRDD = spark.time(query_RDD.collect())

In [ ]:
val timeOptimizedRDD = spark.time(query_optimized_RDD.collect())

In [ ]:
val timeDS = spark.time(query_DS.collect)

In [22]:
query_DS.collect

csv at cmd11.sc:3

1 / 1

collect at cmd21.sc:1

2 / 2

collect at cmd21.sc:1

...

collect at cmd21.sc:1

8 / 8

collect at cmd21.sc:1

8 / 8

res21: Array[(String, Double)] = Array(
  ("Vanuatu", 0.029411764705882353),
  ("Anguilla", 0.03802281368821293),
  ("Montserrat", 0.048507462686567165),
  ("Wallis_and_Futuna", 0.05084745762711865),
  ("Greenland", 0.07037037037037037),
  ("Falkland_Islands_(Malvinas)", 0.07450980392156863),
  ("Marshall_Islands", 0.0851063829787234),
  ("Holy_See", 0.09454545454545454),
  ("Saint_Kitts_and_Nevis", 0.10227272727272728),
  ("Timor_Leste", 0.11567164179104478),
  ("New_Caledonia", 0.13382899628252787),
  ("Laos", 0.15471698113207547),
  ("Fiji", 0.17037037037037037),
  ("Grenada", 0.25842696629213485),
  ("Solomon_Islands", 0.2833333333333333),
  ("British_Virgin_Islands", 0.2889733840304182),
  ("Dominica", 0.3258426966292135),
  ("Saint_Vincent_and_the_Grenadines", 0.3684210526315789),
  ("Northern_Mariana_Islands", 0.4362934362934363),
  ("Brunei_Darussalam", 0.5448028673835126),
  ("Antigua_and_Barbuda", 0.5481481481481482),
  ("Bonaire, Saint Eustatius and Saba", 0.6731517509727627

In [25]:
query_DF.collect

csv at cmd13.sc:3

1 / 1

collect at cmd24.sc:1

2 / 2

collect at cmd24.sc:1

...

collect at cmd24.sc:1

8 / 8

collect at cmd24.sc:1

8 / 8

res24: Array[Row] = Array(
  [Vanuatu,0.029411764705882353],
  [Anguilla,0.03802281368821293],
  [Montserrat,0.048507462686567165],
  [Wallis_and_Futuna,0.05084745762711865],
  [Greenland,0.07037037037037037],
  [Falkland_Islands_(Malvinas),0.07450980392156863],
  [Marshall_Islands,0.0851063829787234],
  [Holy_See,0.09454545454545454],
  [Saint_Kitts_and_Nevis,0.10227272727272728],
  [Timor_Leste,0.11567164179104478],
  [New_Caledonia,0.13382899628252787],
  [Laos,0.15471698113207547],
  [Fiji,0.17037037037037037],
  [Grenada,0.25842696629213485],
  [Solomon_Islands,0.2833333333333333],
  [British_Virgin_Islands,0.2889733840304182],
  [Dominica,0.3258426966292135],
  [Saint_Vincent_and_the_Grenadines,0.3684210526315789],
  [Northern_Mariana_Islands,0.4362934362934363],
  [Brunei_Darussalam,0.5448028673835126],
  [Antigua_and_Barbuda,0.5481481481481482],
  [Bonaire, Saint Eustatius and Saba,0.6731517509727627],
  [Seychelles,0.68],
  [Saint_Lucia,1.0],
  [Cambodia,1.0527859237536656],
 

In [ ]:
val timeDF = spark.time(query_DF.collect)

o bien el framework del cern que nos da más información

In [ ]:
ch.cern.sparkmeasure.StageMetrics(spark).runAndMeasure(query_RDD.collect())

In [ ]:
ch.cern.sparkmeasure.StageMetrics(spark).runAndMeasure(query_optimized_RDD.collect())

In [ ]:
ch.cern.sparkmeasure.StageMetrics(spark).runAndMeasure(query_DS.collect)

In [ ]:
ch.cern.sparkmeasure.StageMetrics(spark).runAndMeasure(query_DF.collect)

# 6. Visualización de datos

In [ ]:
val (x,y) = query_DF
    .collect
    .map(r=>(r(0).toString, r(1).toString.toDouble))
    .toList
    .unzip
Bar(x, y).plot()